In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import skimage.io
import os
from IPython.display import Image

In [ ]:
#Load in image data
arrays = []
directory_ls = []

ugly_count = 0
pretty_count = 0

directory = r'../data/photos'
for root, subdirectories, files in os.walk(directory):
    for file in files:
        if file.endswith('.txt') or file.endswith('json') or file.endswith('.DS_Store'):
          continue
        arrays.append(skimage.io.imread(os.path.join(root, file)))
        directory_ls.append(os.path.join(root, file))
        if 'pretty' in os.path.join(root, file):
          pretty_count+=1
        elif 'ugly' in os.path.join(root, file):
          ugly_count+=1

master_data = np.stack(arrays)
master_data = master_data / 255.0

#Generate labels
labels = []
for i in range(len(master_data)):
  if i >= ugly_count: 
    labels.append([1])
  else:
    labels.append([0])

labels = np.array(labels)
#Cross validation sets
#train_images, test_images, train_labels, test_labels = train_test_split(master_data, labels, test_size=0.33, random_state=42)

In [ ]:
#Shuffles between generating a feature map and then selecting the most prominent features
#This is a pretty standard CNN setup
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
#Add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

model.summary()

In [ ]:
#Compile and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(master_data, labels, epochs=10)

In [ ]:
#Turn logits into interpretable probabilities 
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
#Save model and convert to .json
probability_model.save('build/model.h5')

!tensorflowjs_converter --input_format keras build/model.h5 saved_correct_js

In [ ]:
#Predict and display results

arrays = []
directory_ls = []

directory = r'' #Add path to directory with images you want to predict on
for root, subdirectories, files in os.walk(directory):
    for file in files:
        if file.endswith('.txt') or file.endswith('json') or file.endswith('.DS_Store'):
          continue
        arrays.append(skimage.io.imread(os.path.join(root, file)))
        directory_ls.append(os.path.join(root, file))

test_data = np.stack(arrays)
test_data = test_data / 255.0

predictions = probability_model.predict(test_data)

#Print nth prediction probabilities and display respective image
for i in range(len(test_data)):
  display(Image(filename=directory_ls[i]))
  print("Prediction: " + str(np.argmax(predictions[i])))

